In [2]:
import pandas as pd
from matplotlib import pyplot as plt
# from matplotlib_venn import venn3, venn3_circles
# from upsetplot import UpSet, from_contents

In [3]:
# 读四个文件
cyto_homo = set(pd.read_csv("/lulabdata3/huangkeyun/zhangys/RNA_locator/motif_analysis/Motif_Comparison_Output/homo_final/homo_cyto_favour_motif_final.tsv", sep="\t", header=None)[0])
ev_homo   = set(pd.read_csv("/lulabdata3/huangkeyun/zhangys/RNA_locator/motif_analysis/Motif_Comparison_Output/homo_final/homo_EV_favour_motif_final.tsv",   sep="\t", header=None)[0])
cyto_mus = set(pd.read_csv("/lulabdata3/huangkeyun/zhangys/RNA_locator/motif_analysis/Motif_Comparison_Output/mus_final/mus_cyto_favour_motif_final.tsv", sep="\t", header=None)[0])
ev_mus  = set(pd.read_csv("/lulabdata3/huangkeyun/zhangys/RNA_locator/motif_analysis/Motif_Comparison_Output/mus_final/mus_EV_favour_motif_final.tsv",  sep="\t", header=None)[0])

# # 用字典形式组织
# contents = {
#     "cyto_homo": cyto_homo,
#     "ev_homo": ev_homo,
#     "cyto_mus": cyto_mus,
#     "ev_mus": ev_mus
# }

# # UpSet plot 可视化交集
# upset_data = from_contents(contents)
# UpSet(upset_data).plot()
# plt.show()


In [4]:
print("cyto ∩ EV:(mus)", cyto_mus & ev_mus)
print("cyto ∩ EV:(mus)", cyto_mus & ev_mus)

cyto ∩ EV:(mus) {'(Snrpa)_(Homo_sapiens)_(RBD_0.98)', '(U2af2)_(Homo_sapiens)_(RBD_1.00)', '(ENSMUSG00000056951)_(Homo_sapiens)_(RBD_0.99)', '(Zc3h10)_(Homo_sapiens)_(RBD_1.00)', '(Cpeb2)_(Homo_sapiens)_(RBD_1.00)', '(Rbm8a)_(Homo_sapiens)_(RBD_1.00)', '(Cpeb4)_(Homo_sapiens)_(RBD_1.00)', '(Samd4)_(Homo_sapiens)_(RBD_0.95)', '(Elavl2)_(Homo_sapiens)_(RBD_1.00)', '(Rbm4b)_(Homo_sapiens)_(RBD_0.99)', '(Tia1)_(Homo_sapiens)_(RBD_1.00)'}
cyto ∩ EV:(mus) {'(Snrpa)_(Homo_sapiens)_(RBD_0.98)', '(U2af2)_(Homo_sapiens)_(RBD_1.00)', '(ENSMUSG00000056951)_(Homo_sapiens)_(RBD_0.99)', '(Zc3h10)_(Homo_sapiens)_(RBD_1.00)', '(Cpeb2)_(Homo_sapiens)_(RBD_1.00)', '(Rbm8a)_(Homo_sapiens)_(RBD_1.00)', '(Cpeb4)_(Homo_sapiens)_(RBD_1.00)', '(Samd4)_(Homo_sapiens)_(RBD_0.95)', '(Elavl2)_(Homo_sapiens)_(RBD_1.00)', '(Rbm4b)_(Homo_sapiens)_(RBD_0.99)', '(Tia1)_(Homo_sapiens)_(RBD_1.00)'}


In [5]:
# 找出重复的RBP（同时在ev_homo和cyto_homo中出现的）
overlap_homo = ev_homo.intersection(cyto_homo)

# 找出ev_homo独有的RBP
ev_homo_unique = ev_homo - cyto_homo

# 找出cyto_homo独有的RBP
cyto_homo_unique = cyto_homo - ev_homo

print(f"重复的RBP数量: {len(overlap_homo)}")
print("\n重复的RBP列表:")
print(sorted(list(overlap_homo)))

print(f"\nEV独有的RBP数量: {len(ev_homo_unique)}")
print("\nEV独有的RBP列表:")
print(sorted(list(ev_homo_unique)))

print(f"\nCyto独有的RBP数量: {len(cyto_homo_unique)}")
print("\nCyto独有的RBP列表:")
print(sorted(list(cyto_homo_unique)))

重复的RBP数量: 10

重复的RBP列表:
['ACO1', 'CPEB2', 'CPEB4', 'ELAVL2', 'RBM4', 'RBM8A', 'SAMD4A', 'SNRPA', 'U2AF2', 'ZC3H10']

EV独有的RBP数量: 16

EV独有的RBP列表:
['(PCBP3)_(Mus_musculus)_(RBD_1.00)', '(SNRPB2)_(Drosophila_melanogaster)_(RBD_0.81)', 'EIF4B', 'Fusip1', 'HNRNPH2', 'LIN28A', 'NONO', 'PABPN1', 'PCBP2', 'PPRC1', 'PTBP1', 'RBM42', 'RBM5', 'RBMY1A1', 'ROD1', 'SRSF9']

Cyto独有的RBP数量: 11

Cyto独有的RBP列表:
['(RBM38)_(Mus_musculus)_(RBD_0.99)', 'BRUNOL5', 'ENOX1', 'FMR1', 'HNRNPC', 'HNRNPCL1', 'RALY', 'RBM41', 'RBM4B', 'TIA1', 'ZNF638']


In [6]:
# 找出重复的RBP（同时在ev_mus和cyto_mus中出现的）
overlap_mus = ev_mus.intersection(cyto_mus)

# 找出ev_mus独有的RBP
ev_mus_unique = ev_mus - cyto_mus

# 找出cyto_mus独有的RBP
cyto_mus_unique = cyto_mus - ev_mus

print(f"重复的RBP数量: {len(overlap_mus)}")
print("\n重复的RBP列表:")
print(sorted(list(overlap_mus)))

print(f"\nEV独有的RBP数量: {len(ev_mus_unique)}")
print("\nEV独有的RBP列表:")
print(sorted(list(ev_mus_unique)))

print(f"\nCyto独有的RBP数量: {len(cyto_mus_unique)}")
print("\nCyto独有的RBP列表:")
print(sorted(list(cyto_mus_unique)))

重复的RBP数量: 11

重复的RBP列表:
['(Cpeb2)_(Homo_sapiens)_(RBD_1.00)', '(Cpeb4)_(Homo_sapiens)_(RBD_1.00)', '(ENSMUSG00000056951)_(Homo_sapiens)_(RBD_0.99)', '(Elavl2)_(Homo_sapiens)_(RBD_1.00)', '(Rbm4b)_(Homo_sapiens)_(RBD_0.99)', '(Rbm8a)_(Homo_sapiens)_(RBD_1.00)', '(Samd4)_(Homo_sapiens)_(RBD_0.95)', '(Snrpa)_(Homo_sapiens)_(RBD_0.98)', '(Tia1)_(Homo_sapiens)_(RBD_1.00)', '(U2af2)_(Homo_sapiens)_(RBD_1.00)', '(Zc3h10)_(Homo_sapiens)_(RBD_1.00)']

EV独有的RBP数量: 18

EV独有的RBP列表:
['(Eif4b)_(Homo_sapiens)_(RBD_0.96)', '(Hnrnph2)_(Homo_sapiens)_(RBD_0.99)', '(Lin28a)_(Homo_sapiens)_(RBD_0.99)', '(Lin28b)_(Homo_sapiens)_(RBD_0.83)', '(Nono)_(Homo_sapiens)_(RBD_1.00)', '(Pabpn1)_(Homo_sapiens)_(RBD_1.00)', '(Pcbp2)_(Homo_sapiens)_(RBD_1.00)', '(Pprc1)_(Homo_sapiens)_(RBD_1.00)', '(Ptbp1)_(Homo_sapiens)_(RBD_0.96)', '(Rbm42)_(Homo_sapiens)_(RBD_1.00)', '(Rbm5)_(Homo_sapiens)_(RBD_1.00)', '(Rod1)_(Homo_sapiens)_(RBD_0.80)', '(Sf3b4)_(Danio_rerio)_(RBD_1.00)', '(Snrpb2)_(Drosophila_melanogaster)_(RBD_0